In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import json
import pandas as pd
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import random

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data"
model_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/model"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/csv"
json_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/json"  # ✅ JSON 저장 경로
os.makedirs(model_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(json_dir, exist_ok=True)  # ✅ JSON 저장 폴더 생성

# ✅ 데이터셋 분할 인덱스 저장 경로
train_indices_path = os.path.join(json_dir, "model1train.json")
val_indices_path = os.path.join(json_dir, "model1val.json")
test_indices_path = os.path.join(json_dir, "model1test.json")

# ✅ 하이퍼파라미터 설정
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 30
LEARNING_RATE = 0.001

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ 각 클래스별 이미지 인덱스 분배
class_indices = {cls: [] for cls in dataset.class_to_idx.values()}

for idx, (path, label) in enumerate(dataset.samples):
    class_indices[label].append(idx)

# ✅ 각 클래스별로 7:2:1 비율로 분할 (학습:검증:테스트)
train_indices, val_indices, test_indices = [], [], []

for indices in class_indices.values():
    random.shuffle(indices)  # 랜덤 셔플
    num_total = len(indices)
    num_train = int(num_total * 0.7)
    num_val = int(num_total * 0.2)
    num_test = num_total - num_train - num_val

    train_indices.extend(indices[:num_train])
    val_indices.extend(indices[num_train:num_train+num_val])
    test_indices.extend(indices[num_train+num_val:])

# ✅ 분할된 인덱스를 JSON 파일로 저장
with open(train_indices_path, "w") as f:
    json.dump(train_indices, f)
with open(val_indices_path, "w") as f:
    json.dump(val_indices, f)
with open(test_indices_path, "w") as f:
    json.dump(test_indices, f)

# ✅ 데이터셋 분할
dataset_train = Subset(dataset, train_indices)
dataset_val = Subset(dataset, val_indices)
dataset_test = Subset(dataset, test_indices)

# ✅ 데이터 로더 설정
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# ✅ EfficientNet-B0 모델 정의
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.to(device)

# ✅ 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ✅ 학습 루프
best_valid_accuracy = 0.0
best_epoch = 0
csv_path = os.path.join(csv_dir, "category_classification_results.csv")
train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []

print("\n📢 Training Started! Logging Every Epoch:\n")

for epoch in range(EPOCHS):
    print(f"\n📢 Epoch [{epoch+1}/{EPOCHS}] 시작")
    model.train()
    train_loss, train_correct = 0, 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()
        total_train += labels.size(0)

    train_loss /= total_train
    train_accuracy = train_correct / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # ✅ 검증 단계
    model.eval()
    valid_loss, valid_correct = 0, 0
    total_valid = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            valid_correct += (outputs.argmax(1) == labels).sum().item()
            total_valid += labels.size(0)

    valid_loss /= total_valid
    valid_accuracy = valid_correct / total_valid
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)

    print(f"   🎯 Train Accuracy: {train_accuracy:.4f} | 📉 Train Loss: {train_loss:.4f} | "
        f"🎯 Valid Accuracy: {valid_accuracy:.4f} | 📉 Valid Loss: {valid_loss:.4f}")

    if valid_accuracy > best_valid_accuracy:
        best_valid_accuracy = valid_accuracy
        best_epoch = epoch + 1
        best_model_path = os.path.join(model_dir, "best_category_model.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"🎯 새로운 최고 모델 저장됨! (Epoch {epoch+1}, Accuracy: {best_valid_accuracy:.4f})")

# ✅ 최고의 검증 정확도를 기록한 에포크 모델 저장
model_save_path = os.path.join(model_dir, "final_category_model.pth")
torch.save(model.state_dict(), model_save_path)
print(f"📄 최고의 모델이 저장되었습니다: {model_save_path}")

# ✅ 결과 CSV 저장
df = pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Valid Loss': valid_losses,
    'Train Accuracy': train_accuracies,
    'Valid Accuracy': valid_accuracies
})
csv_path = os.path.join(csv_dir, "category_classification_results.csv")
df.to_csv(csv_path, index=False)
print(f"📄 학습 결과 CSV 저장됨: {csv_path}")

# ✅ 검증 정확도가 가장 높은 에포크의 모든 정보 출력
print(f"✅ 학습 종료! 최고 성능을 기록한 Epoch: {best_epoch}, 🎯 Train Accuracy: {train_accuracies[best_epoch-1]:.4f}, 📉 Train Loss: {train_losses[best_epoch-1]:.4f}, 🎯 Valid Accuracy: {best_valid_accuracy:.4f}, 📉 Valid Loss: {valid_losses[best_epoch-1]:.4f}")


✅ 사용 중인 GPU: Quadro RTX 4000

📢 Training Started! Logging Every Epoch:


📢 Epoch [1/30] 시작
   🎯 Train Accuracy: 0.7248 | 📉 Train Loss: 0.8655 | 🎯 Valid Accuracy: 0.7976 | 📉 Valid Loss: 0.6001
🎯 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 0.7976)

📢 Epoch [2/30] 시작
   🎯 Train Accuracy: 0.8387 | 📉 Train Loss: 0.4997 | 🎯 Valid Accuracy: 0.8164 | 📉 Valid Loss: 0.6194
🎯 새로운 최고 모델 저장됨! (Epoch 2, Accuracy: 0.8164)

📢 Epoch [3/30] 시작
   🎯 Train Accuracy: 0.8801 | 📉 Train Loss: 0.3607 | 🎯 Valid Accuracy: 0.8463 | 📉 Valid Loss: 0.5667
🎯 새로운 최고 모델 저장됨! (Epoch 3, Accuracy: 0.8463)

📢 Epoch [4/30] 시작
   🎯 Train Accuracy: 0.9136 | 📉 Train Loss: 0.2660 | 🎯 Valid Accuracy: 0.8319 | 📉 Valid Loss: 0.5880

📢 Epoch [5/30] 시작
   🎯 Train Accuracy: 0.9278 | 📉 Train Loss: 0.2143 | 🎯 Valid Accuracy: 0.8485 | 📉 Valid Loss: 0.5894
🎯 새로운 최고 모델 저장됨! (Epoch 5, Accuracy: 0.8485)

📢 Epoch [6/30] 시작
   🎯 Train Accuracy: 0.9366 | 📉 Train Loss: 0.1921 | 🎯 Valid Accuracy: 0.8561 | 📉 Valid Loss: 0.5549
🎯 새로운 최고 모델 저장됨! (Epoch 6, A

In [11]:
import os
import torch
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import json
import pandas as pd

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0))

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data"
model_path = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\model\best_category_model.pth"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/csv"
json_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/json"  # ✅ JSON 저장 경로
os.makedirs(csv_dir, exist_ok=True)

# ✅ 데이터셋 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ JSON에서 인덱스 로드
def load_indices(json_path):
    with open(json_path, "r") as f:
        return json.load(f)

train_indices = load_indices(os.path.join(json_dir, "model1train.json"))
val_indices = load_indices(os.path.join(json_dir, "model1val.json"))
test_indices = load_indices(os.path.join(json_dir, "model1test.json"))

# ✅ 데이터셋 분할
dataset_train = Subset(dataset, train_indices)
dataset_val = Subset(dataset, val_indices)
dataset_test = Subset(dataset, test_indices)

# ✅ 데이터 로더 설정
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
valid_loader = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 불러오기
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

# ✅ 평가 함수 정의
def evaluate_model(loader, dataset_name):
    correct = 0
    total = 0
    results = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(labels)):
                image_idx = idx * loader.batch_size + i
                pred_label = dataset.classes[predicted[i].item()]
                true_label = dataset.classes[labels[i].item()]
                if pred_label == true_label:
                    results.append(f"[✅ 정답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
                else:
                    results.append(f"[❌ 오답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
    
    accuracy = correct / total
    print(f"✅ {dataset_name} 데이터셋 정확도: {accuracy:.4f}")
    return accuracy, results

# ✅ 평가 수행
train_accuracy, train_results = evaluate_model(train_loader, "Train")
valid_accuracy, valid_results = evaluate_model(valid_loader, "Valid")
test_accuracy, test_results = evaluate_model(test_loader, "Test")

# ✅ 정확도 및 리콜 값 CSV 저장
df = pd.DataFrame({
    'Dataset': ['Train', 'Valid', 'Test'],
    'Accuracy': [train_accuracy, valid_accuracy, test_accuracy]
})

csv_path = os.path.join(csv_dir, "category_classification_evaluation.csv")
df.to_csv(csv_path, index=False)
print(f"📄 평가 결과 CSV 저장됨: {csv_path}")

# ✅ 상세 결과 저장
detail_results_path = os.path.join(csv_dir, "detailed_classification_results.txt")
with open(detail_results_path, "w", encoding="utf-8") as f:
    f.write("\n".join(train_results + valid_results + test_results))
print(f"📄 상세 평가 결과 저장됨: {detail_results_path}")


✅ CUDA Available: True
✅ 사용 중인 GPU: Quadro RTX 4000
✅ Train 데이터셋 정확도: 0.9954
✅ Valid 데이터셋 정확도: 0.8618
✅ Test 데이터셋 정확도: 0.8734
📄 평가 결과 CSV 저장됨: C:/Users/user/OneDrive/Desktop/Resnet182-real/csv\category_classification_evaluation.csv
📄 상세 평가 결과 저장됨: C:/Users/user/OneDrive/Desktop/Resnet182-real/csv\detailed_classification_results.txt


In [12]:
import os
import shutil
import torch
import json
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data"
model_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/model"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/csv"
json_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/json"
wrong_images_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/wrong_images"  # 틀린 이미지를 저장할 경로

# 새로운 디렉토리 생성
os.makedirs(wrong_images_dir, exist_ok=True)

# ✅ 데이터셋 로드
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ JSON에서 인덱스 로드
def load_indices(json_path):
    with open(json_path, "r") as f:
        return json.load(f)

train_indices = load_indices(os.path.join(json_dir, "model1train.json"))
val_indices = load_indices(os.path.join(json_dir, "model1val.json"))
test_indices = load_indices(os.path.join(json_dir, "model1test.json"))

# ✅ 데이터셋 분할
dataset_train = Subset(dataset, train_indices)
dataset_val = Subset(dataset, val_indices)
dataset_test = Subset(dataset, test_indices)

# ✅ 데이터 로더 설정
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
valid_loader = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 불러오기
model_path = os.path.join(model_dir, "best_category_model.pth")  # 또는 "final_category_model.pth"
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

# ✅ 평가 함수 정의
def evaluate_model(loader, dataset_name):
    correct = 0
    total = 0
    wrong_images = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # 각 이미지에 대해 정답과 예측값 비교
            for i in range(len(labels)):
                img_path = loader.dataset.dataset.samples[idx * loader.batch_size + i][0]
                true_label = dataset.classes[labels[i].item()]
                pred_label = dataset.classes[predicted[i].item()]

                # 틀린 이미지 정보 추출
                if true_label != pred_label:
                    wrong_images.append((pred_label, true_label, img_path))

    accuracy = correct / total
    print(f"✅ {dataset_name} 데이터셋 정확도: {accuracy:.4f}")
    return accuracy, wrong_images

# ✅ 모델 평가
train_accuracy, train_wrong_images = evaluate_model(train_loader, "Train")
valid_accuracy, valid_wrong_images = evaluate_model(valid_loader, "Valid")
test_accuracy, test_wrong_images = evaluate_model(test_loader, "Test")

# ✅ 틀린 이미지를 새로운 폴더에 저장 (원본 폴더 구조를 유지하며 복사)
def save_wrong_images(wrong_images, base_dir, save_dir):
    for pred_label, true_label, img_path in wrong_images:
        # 원본 폴더에서 이미지 경로 추출
        class_folder = os.path.basename(os.path.dirname(img_path))
        new_dir = os.path.join(save_dir, class_folder)
        os.makedirs(new_dir, exist_ok=True)

        # 이미지 복사
        shutil.copy(img_path, new_dir)

# ✅ 각 데이터셋에서 틀린 이미지 저장
save_wrong_images(train_wrong_images, data_dir, os.path.join(wrong_images_dir, "train"))
save_wrong_images(valid_wrong_images, data_dir, os.path.join(wrong_images_dir, "valid"))
save_wrong_images(test_wrong_images, data_dir, os.path.join(wrong_images_dir, "test"))

print(f"📄 틀린 이미지가 저장되었습니다 (train, valid, test): {wrong_images_dir}")

# ✅ 결과 CSV 저장
df = pd.DataFrame({
    'Dataset': ['Train', 'Valid', 'Test'],
    'Accuracy': [train_accuracy, valid_accuracy, test_accuracy]
})

csv_path = os.path.join(csv_dir, "category_classification_evaluation.csv")
df.to_csv(csv_path, index=False)
print(f"📄 평가 결과 CSV 저장됨: {csv_path}")


✅ 사용 중인 GPU: Quadro RTX 4000
✅ Train 데이터셋 정확도: 0.9954
✅ Valid 데이터셋 정확도: 0.8618
✅ Test 데이터셋 정확도: 0.8734
📄 틀린 이미지가 저장되었습니다 (train, valid, test): C:/Users/user/OneDrive/Desktop/Resnet182-real/wrong_images
📄 평가 결과 CSV 저장됨: C:/Users/user/OneDrive/Desktop/Resnet182-real/csv\category_classification_evaluation.csv
